#Realizar un embedding para el campo text y visualizarlo con alguna técnica de reducción de visualizaciones distinta a PCA. En base a la visualización,

##¿Sería posible predecir el target con este embedding? (2 punto)

In [1]:
# @title 1. Instalación de Librerías e Importaciones

!pip install -U gensim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.tokenize import TweetTokenizer
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 50.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# @title Semilla y Carga de Datos

SEMILLA = 111014

# Cargar datos
train_file = '/content/drive/MyDrive/MATERIAS/Ciencia de Datos/nlp-getting-started/train.csv'
df_train = pd.read_csv(train_file)

print(f"Datos cargados: {df_train.shape}")

Datos cargados: (7613, 5)


In [3]:
# @title 3. Preprocesamiento (Tokenización)

tk = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def preprocesar_tweet(text):
    if pd.isna(text): return []
    tokens = tk.tokenize(text)
    tokens = [token for token in tokens if token.isalnum() or token.startswith('#')]

    return tokens

df_train['tokens'] = df_train['text'].apply(preprocesar_tweet)

In [4]:
# @title 4. Entrenamiento Word2Vec y Vectorización
w2v_model = Word2Vec(
    sentences=df_train['tokens'],
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,
    seed=SEMILLA,
    workers=4
)

def obtener_vector_promedio(tokens, modelo):
    palabras_validas = [word for word in tokens if word in modelo.wv]

    if len(palabras_validas) > 0:
        return np.mean(modelo.wv[palabras_validas], axis=0)
    else:
        return np.zeros(modelo.vector_size)

embeddings = np.array([obtener_vector_promedio(t, w2v_model) for t in df_train['tokens']])

In [5]:
# @title 5. Reducción con t-SNE y Visualización

tsne = TSNE(
    n_components=2,
    perplexity=80,
    random_state=SEMILLA,
    learning_rate='auto',
    n_jobs=-1
)

embeddings_2d = tsne.fit_transform(embeddings)

In [6]:

# 2. Preparar datos para el gráfico
df_viz = df_train.copy()
df_viz['x'] = embeddings_2d[:, 0]
df_viz['y'] = embeddings_2d[:, 1]
df_viz['clase'] = df_viz['target'].map({0: 'No Desastre', 1: 'Desastre'})


In [7]:
# @title 6. Visualización Interactiva con Plotly
import plotly.express as px
import pandas as pd

df_tsne = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
df_tsne['target_label'] = df_train['target'].map({0: 'No Desastre', 1: 'Desastre'})
df_tsne['text'] = df_train['text']
df_tsne['keyword'] = df_train['keyword'].fillna('Sin Keyword')

fig = px.scatter(
    df_tsne,
    x='x',
    y='y',
    color='target_label',

    hover_data={
        'x': False,
        'y': False,
        'target_label': True,
        'keyword': True,
        'text': True
    },

    title='Embedding de Tweets usando Word2Vec + T-SNE',

    color_discrete_map={'No Desastre': '#808080', 'Desastre': '#C0392B'},

    opacity=0.6,
    width=1000,
    height=800
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    legend_title_text='Clase Real',
    template='plotly_white'
)

fig.show()

# ¿Sería posible predecir el target con este embedding?

### Con este embedding seria un poco dificil definir el target, pero podemos ver que hay zonas en las que predomina un target más que otro.

Podemos ver que la zona de la izquierda tiene mas Tweets que son de NO Desastres y la zona más de la derecha son de Desastres.
Tambien podemos identificar clusters que hablan de lo mismo por ejemplo de un choque de un avion o cosas como esas.

# Y si usamos un embedding mejor?
 - Se nombro Bert que es un embedding mucho mejor, vamos a probarlo
https://huggingface.co/docs/transformers/main/model_doc/bert

In [ ]:
# @title Semilla y Carga de Datos

SEMILLA = 111014

# Cargar datos
train_file = '/content/drive/MyDrive/MATERIAS/Ciencia de Datos/nlp-getting-started/train.csv'
df_train = pd.read_csv(train_file)

print(f"Datos cargados: {df_train.shape}")

Datos cargados: (7613, 5)


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import plotly.express as px


model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df_train['text'].tolist(), show_progress_bar=True)

tsne = TSNE(n_components=2, random_state=SEMILLA, perplexity=30, n_iter=1000, n_jobs=-1)
bert_2d = tsne.fit_transform(embeddings)

df_bert_viz = pd.DataFrame(bert_2d, columns=['x', 'y'])
df_bert_viz['target_label'] = df_train['target'].map({0: 'No Desastre', 1: 'Desastre'})
df_bert_viz['text'] = df_train['text']
df_bert_viz['keyword'] = df_train['keyword'].fillna('Sin Keyword')
df_bert_viz['location'] = df_train['location'].fillna('Sin Location')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/238 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [ ]:

# Gráfico interactivo
fig = px.scatter(
    df_bert_viz,
    x='x',
    y='y',
    color='target_label',
    hover_data={
        'x': False,
        'y': False,
        'target_label': True,
        'keyword': True,
        'location': True,
        'text': True
    },
    title='Mapa Semántico de Tweets (BERT + t-SNE)',
    color_discrete_map={'No Desastre': '#808080', 'Desastre': '#C0392B'},
    opacity=0.6,
    width=1000,
    height=800
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    legend_title_text='Clase Real',
    template='plotly_white'
)

fig.show()

# En este Embedding, se puede ver más claramente la diferencia del target y se podria predecir con mayor precision.
## Tambien al igual que antes, podemos identificar clusters del Mismo Target que hablan del mismo tema, lo que nos dice que es un buen Embedding